In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data load

In [31]:
try:
    data = pd.read_csv("data/processed/NASDAQ_FULL_2021_Cleaned_Imputed.csv")
    print(f"Data loaded successfully with {data.shape[0]} rows (days) and {data.shape[1]} assets.")
except FileNotFoundError:
    print("Error: file NASDAQ_FULL_2021_Cleaned_Imputed.csv not found.")

Data loaded successfully with 252 rows (days) and 6540 assets.


In [32]:
data.head()

,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMD,AMGN,AMT,ASML,...,GRU,GSCE,GSP,HDLB,HMDCF,HOMU,HWCPZ,HAWLM,HBANP,HFRO-PA
0,125.974464,86.739708,99.687309,238.710358,485.339996,153.288055,92.300003,193.642502,191.466125,477.949158,...,3.5100,17.25,10.920,6.557615,5.1097,0.55,19.842388,15.62887,18.716362,19.115032
1,127.532021,87.636642,100.920723,240.069351,485.690002,152.861603,92.769997,194.582230,192.433868,483.072815,...,3.4950,17.25,11.430,6.720678,5.1097,0.55,19.909050,15.62887,18.716362,19.110727
2,123.239037,86.879608,100.710602,242.694153,466.309998,152.326324,90.330002,199.272491,187.359482,468.600525,...,3.5450,17.25,11.465,7.058459,5.1097,0.55,19.716475,15.62887,18.716362,19.087008
3,127.444374,87.809464,101.688164,244.983932,477.739990,153.188248,95.160004,199.930344,189.635117,479.316132,...,3.4501,17.25,11.490,7.087579,5.1097,0.55,19.894239,15.62887,18.716362,19.116274
4,128.544388,88.270248,101.971436,245.877502,485.100006,155.184143,94.580002,203.749222,193.733017,486.303741,...,3.5650,17.25,11.690,7.081756,5.1097,0.55,19.975708,15.62887,18.716362,19.026497


### Log returns

In [33]:
eps = 1e-4

# np.log(P_t / P_{t-1}).
log_returns = np.log((data / data.shift(1)) + eps)

# Drop the first row which contains NaN values (as there is no P_{t-1} for the first day)
log_returns = log_returns.dropna(how='all')

print(f"Logarithmic returns calculated. Number of observations: {log_returns.shape[0]}.")

Logarithmic returns calculated. Number of observations: 251.
